# Simple CNN for Image Classification
-  dataset: MNIST
- TensorFlow: tf.nn, tf.Estimator
- Colab TensorBoard monitoring

In [0]:
from datetime import datetime
import os
import tensorflow as tf
import numpy as np

In [2]:
now = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
MODEL_DIR=os.path.join("/tmp/mnist_convnet_model", now)

try:  
    os.makedirs(MODEL_DIR)
except OSError:  
    print ("Creation of the directory %s failed" % MODEL_DIR)
else:  
    print ("Successfully created the directory %s " % MODEL_DIR)

Successfully created the directory /tmp/mnist_convnet_model/20190507_150809 


## Start TensorBoard

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(MODEL_DIR)
)

##[optional] Get ngrock and run it


In [4]:
import requests, time

max_tries=3  # max attempts
grace_time=1 # seconds

![[ ! -f "ngrok" ]]  \
  && wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip \
  && unzip ngrok-stable-linux-amd64.zip \
  && rm ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')
time.sleep(grace_time)

for attempt in range(max_tries):
  try:
    tunnels_list = requests.get('http://localhost:4040/api/tunnels').json()['tunnels']
    print('resource available at: {}'.format(tunnels_list[0]['public_url']))
    break
    
  except ConnectionRefusedError:
    print('retry after {} second[s]'.format(grace_time))
    time.sleep(grace_time)  

resource available at: https://6309c34a.ngrok.io


# Define Estimator functions

In [0]:
#tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
def make_model_fn(optimizer, cnn_config, fc_config, 
                  input_shape = (28,28,1), classes = 10): 
                  
  [ HEIGHT, WIDTH, CHANNELS ] = input_shape
  
  def net_summary(net):
    for idx, t in enumerate(net):
      print('{} {} {}'.format(idx+1, t.name, t.shape))
  
  def _add_fc(net, config):
    for hidden in config:
      with tf.variable_scope('dense'):
        net.append(tf.keras.layers.Dense(hidden, 
                                         activation='relu', 
                                         name='fc')(net[-1]))
    return net
  
  def _add_cnn(net, config):
    for k, s, f in config:
      with tf.variable_scope('conv'):
        
        last_layer = net[-1]
        kernel = tf.Variable(tf.truncated_normal(shape=(k, k, last_layer.shape[-1].value, f), 
                                                 dtype=tf.float32, mean=0,
                                                 stddev=0.1), name='kernel')
        net.append(tf.nn.conv2d(last_layer, kernel, strides=(1, 1, 1, 1),
                            padding='SAME', name='conv'))
        
        net.append(tf.nn.relu(net[-1], name='relu'))
        
        net.append(tf.nn.max_pool(net[-1], ksize=(1, s, s, 1),
                              strides=(1, s, s, 1), padding='SAME',
                              name='pool'))
    
    return net
    
  def _model_fn(features, labels, mode, params=None):
    """Model function for CNN."""
    
    net = []
    
    # Input Layer
    net.append(tf.reshape(features["x"], [-1, HEIGHT, WIDTH, CHANNELS], name='input'))

    # Convolutional Layers
    net = _add_cnn(net, cnn_config)
    
    # Reshape
    embedding_len = np.prod(net[-1].shape.as_list()[1:])
    net.append(tf.reshape(net[-1],[-1, embedding_len]))
    
    # Dense Layers   
    net = _add_fc(net, fc_config)
    
    # Dropout layer (on train graph)
    net.append(tf.layers.dropout(inputs=net[-1], rate=0.4,
                                 training=mode == tf.estimator.ModeKeys.TRAIN))
    
    # Logits Layer
    net.append(tf.keras.layers.Dense(classes, name='logits')(net[-1]))
    logits = net[-1]
    
    net_summary(net)
    
    # Compute predictions
    predicted_classes = tf.argmax(input=logits, axis=1)
    
    predictions = {
        'classes': predicted_classes[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits, name="softmax_tensor"),
        'logits': logits,
    }
    
    ###########
    # PREDICT #
    ###########
    if mode == tf.estimator.ModeKeys.PREDICT:
      return tf.estimator.EstimatorSpec(mode=mode,
                                        predictions=predictions)

    # Loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,
                                                  logits=logits)

    # Metrics
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    
    ########
    # EVAL #
    ########
    if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode,
                                        loss=loss, 
                                        eval_metric_ops=metrics)
    
    #########
    # TRAIN #
    #########
    assert mode == tf.estimator.ModeKeys.TRAIN
    
    train_op = optimizer.minimize(loss, 
                                  global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
  
  return _model_fn

In [0]:
def make_train_input_fn(features,
                        labels,
                        batch_size=100,
                        num_epochs=None,
                        shuffle=False):

  train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": features},
                                                      y=labels,
                                                      batch_size=batch_size,
                                                      num_epochs=num_epochs,
                                                      shuffle=shuffle)
  return train_input_fn

def make_eval_input_fn(features,
                       labels):
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": features},
                                                     y=labels,
                                                     num_epochs=1,
                                                     shuffle=False)
  
  return eval_input_fn

# Load Dataset

In [0]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

# Create the Estimator

In [0]:
learning_rate=0.1
batch_size=100
max_steps=20000

In [10]:
# Create the Estimator
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 2000,
    #keep_checkpoint_max = 10,
    log_step_count_steps = 500,
    tf_random_seed=2020         # for reproducibility
)

train_spec = tf.estimator.TrainSpec(
    input_fn=make_train_input_fn(train_data,
                                 train_labels,
                                 batch_size=batch_size),
    max_steps=max_steps)

eval_spec = tf.estimator.EvalSpec(
    input_fn=make_eval_input_fn(eval_data,
                                eval_labels),
    steps=None,          # use complete eval set
    start_delay_secs=0,  # start immediately
    throttle_secs=10)    # minimum delay between evaluations

cnn_config = [ [5, 2, 32], [5, 2, 64] ]
fc_config = [ 1024 ]
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
classifier = tf.estimator.Estimator(
  model_fn=make_model_fn(optimizer, cnn_config, fc_config),
  model_dir=MODEL_DIR,
  config=my_checkpointing_config
)


INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model/20190507_150809', '_tf_random_seed': 2020, '_save_summary_steps': 100, '_save_checkpoints_steps': 2000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb281b624e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Start Training

In [11]:
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 2000 or save_checkpoints_secs None.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
1 input:0 (100, 28, 28, 1)
2 conv/conv:0 (100, 28, 28, 32)
3 conv/relu:0 (100, 28, 28, 32)
4 conv/pool:0 (100, 14, 14, 32)
5 conv_1/conv:0 (100, 14, 14, 64)
6 conv_1/relu:0 (100, 14, 14, 64)
7 conv_1/pool:0 

({'accuracy': 0.9941, 'global_step': 20000, 'loss': 0.027606478}, [])

# [optional] Stop Tensorboard

In [0]:
#!pkill tensorboard
#!pkill ngrok